### Scraping of Titles

In [6]:
import praw
import numpy as np
import pandas as pd
from praw.models import MoreComments

reddit_read_only = praw.Reddit(client_id="",         # your client id
                               client_secret="",      # your client secret
                               user_agent="")        # your user agent

In [7]:
subreddit = reddit_read_only.subreddit("MomForAMinute")

posts = subreddit.top(time_filter='year', limit = 5000)
# Scraping the top posts of the current month
 
postsDict = {"Title": [], "Post Text": [],
              "ID": [], "Score": [],
              "Total Comments": [], "Post URL": []
              }
 
for post in posts:
    # Title of each post
    postsDict["Title"].append(post.title)
     
    # Text inside a post
    postsDict["Post Text"].append(post.selftext)
     
    # Unique ID of each post
    postsDict["ID"].append(post.id)
     
    # The score of a post
    postsDict["Score"].append(post.score)
     
    # Total number of comments inside the post
    postsDict["Total Comments"].append(post.num_comments)
     
    # URL of each post
    postsDict["Post URL"].append(post.url)
 
# Saving the data in a pandas dataframe
topPosts = pd.DataFrame(postsDict)
topPosts

,Title,Post Text,ID,Score,Total Comments,Post URL
0,My daughter passed away when she was 15. Today...,,shxdas,13138,828,https://i.redd.it/uieiwvhnn8f81.jpg
1,Mom is the prom dress I choose really that bad...,,tl4uwe,12492,2613,https://www.reddit.com/gallery/tl4uwe
2,Hey Mom! So I’ve reached a pretty big mileston...,My Mom passed away before I was able to come o...,weu7zn,8962,531,https://i.redd.it/3ux0bpvgref91.jpg
3,"Mom here, if you need support or help camping ...",,vjq0sl,8262,349,https://i.redd.it/1sjfm6dlzk791.jpg
4,"Mom, I did it. I’m alive. I rang that MF bell.",,upwgxi,6959,188,https://v.redd.it/kdyzeaczyjz81
...,...,...,...,...,...,...
991,I am on day 3 of sobriety,I could really use some words of encouragement...,z6qrvx,261,44,https://www.reddit.com/r/MomForAMinute/comment...
992,Hey mom- I picked up my one year chip.,I picked up my one year chip in my CoDA group ...,th23yx,263,37,https://www.reddit.com/r/MomForAMinute/comment...
993,I finally came out as trans today to safe peop...,"Hey mom,\n\nI finally came out as trans today ...",yx7ein,259,16,https://www.reddit.com/r/MomForAMinute/comment...
994,I just need someone to tell me my baby mattered.,TW- Miscarriage \n\nToday is Pregnancy and Chi...,y55h67,263,39,https://www.reddit.com/r/MomForAMinute/comment...


In [8]:
# Limiting to top 100 title
top100 = topPosts.head(100)

### Scraping of Comments

In [9]:
# Dictionary with title id and corresponding URL
urlDict = {top100["ID"][i]: top100["Post URL"][i] for i in range(len(top100["ID"]))}

# Formatting for comment dictionary
commentDict = {"Post ID": [], "User": [], "Body": [], "Score": []}

# Pulling comments from each title
for id in urlDict:
    submission = reddit_read_only.submission(id)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        commentDict["Post ID"].append(id)
        commentDict["User"].append(comment.author)
        commentDict["Body"].append(comment.body)
        commentDict["Score"].append(comment.score)

allComments = pd.DataFrame(commentDict)
allComments

,Post ID,User,Body,Score
0,shxdas,callme_rdubs,"No, but if you need a kid to tell you that you...",1722
1,shxdas,asghettimonster,Your picture is absolutely beautiful. xoxo,433
2,shxdas,Usernamenottaken13,"Hi, mom. I hate leaving my apartment, and I ha...",363
3,shxdas,hgshowal,We love you mom!! You have a whole family here...,329
4,shxdas,iwannabeunknown3,"I am in a decent place, but I can adopt a mom ...",600
...,...,...,...,...
22129,vufta1,apoohneicie,Blech. We can just agree to disagree on provol...,1
22130,vufta1,leavetaterualone,"Nope, now you have to like Swiss or face the w...",1
22131,vufta1,apoohneicie,I will die first!!!,2
22132,vufta1,AlwaysSleepingLolX2,"Swiss is awful, Gouda sucks.",-1


### Combining Titles & Comments

In [10]:
# Rename Post ID column to match name in topPosts & score column
allComments = allComments.rename(columns= {"Post ID": "ID", "Score": "Comment Score"})

# Rename score column in topPosts
top100 = top100.rename(columns= {"Score": "Title Score"})

# Combining topPosts and allComments dataframes
combinedScrape = pd.merge(top100, allComments, on= "ID")

In [11]:
combinedScrape

,Title,Post Text,ID,Title Score,Total Comments,Post URL,User,Body,Comment Score
0,My daughter passed away when she was 15. Today...,,shxdas,13138,828,https://i.redd.it/uieiwvhnn8f81.jpg,callme_rdubs,"No, but if you need a kid to tell you that you...",1722
1,My daughter passed away when she was 15. Today...,,shxdas,13138,828,https://i.redd.it/uieiwvhnn8f81.jpg,asghettimonster,Your picture is absolutely beautiful. xoxo,433
2,My daughter passed away when she was 15. Today...,,shxdas,13138,828,https://i.redd.it/uieiwvhnn8f81.jpg,Usernamenottaken13,"Hi, mom. I hate leaving my apartment, and I ha...",363
3,My daughter passed away when she was 15. Today...,,shxdas,13138,828,https://i.redd.it/uieiwvhnn8f81.jpg,hgshowal,We love you mom!! You have a whole family here...,329
4,My daughter passed away when she was 15. Today...,,shxdas,13138,828,https://i.redd.it/uieiwvhnn8f81.jpg,iwannabeunknown3,"I am in a decent place, but I can adopt a mom ...",600
...,...,...,...,...,...,...,...,...,...
22129,Today I cooked for the first time in a year,,vufta1,2259,87,https://i.redd.it/r3kqupor5fa91.jpg,apoohneicie,Blech. We can just agree to disagree on provol...,1
22130,Today I cooked for the first time in a year,,vufta1,2259,87,https://i.redd.it/r3kqupor5fa91.jpg,leavetaterualone,"Nope, now you have to like Swiss or face the w...",1
22131,Today I cooked for the first time in a year,,vufta1,2259,87,https://i.redd.it/r3kqupor5fa91.jpg,apoohneicie,I will die first!!!,2
22132,Today I cooked for the first time in a year,,vufta1,2259,87,https://i.redd.it/r3kqupor5fa91.jpg,AlwaysSleepingLolX2,"Swiss is awful, Gouda sucks.",-1


In [12]:
# Converting dataframes to csv
topPosts.to_csv("ScrapedTitles.csv")
top100.to_csv("Top100ScrapedTitles.csv")
allComments.to_csv("ScrapedComments.csv")
combinedScrape.to_csv("CombinedTop100Scrape.csv")